# Graph Search, Parameterized on Path Data Structures

In [ ]:
graph = {'A': ['B', 'C'],
         'B': ['C', 'D'],
         'C': ['D'],
         'D': ['C'],
         'E': ['F'],
         'F': ['C']}

In [ ]:
class Path:
    def __init__(self, path=[]):
        self.path = path

    def add(self, node):
        return Path(self.path + [node])

    def visited(self, node):
        return node in self.path

    def len(self):
        return len(self.path)

    def as_list(self):
        return self.path

In [ ]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

def find_path_main(graph, start, end):
    p = find_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [ ]:
find_path_main(graph, 'A', 'D')

In [ ]:
def find_all_paths(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if not path.visited(node):
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

def find_all_paths_main(graph, start, end):
    return [p.as_list()
            for p in find_all_paths(graph, start, end, Path())]


def find_shortest_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if start not in graph:
        return None
    shortest = None
    for node in graph[start]:
        if not path.visited(node):
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or newpath.len() < shortest.len():
                    shortest = newpath
    return shortest

def find_shortest_path_main(graph, start, end):
    p = find_shortest_path(graph, start, end, Path())
    if p:
        return p.as_list()
    else:
        return p

In [ ]:
find_all_paths_main(graph, 'A', 'D')

In [ ]:
find_shortest_path_main(graph, 'A', 'D')

In [ ]:
# To see how inefficient this pathfinding is, let's use this trivial straight-line graph.
def straight_line(n):
    return {i: [i+1] for i in range(n)}

# Python's limit on recursion means we'll need to run several searches to get the times high enough to compare.
def perf_test(n, path):
    x = [find_path(straight_line(n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test(1000, Path())

## Linked-List Paths

In [ ]:
class EmptyLinkedPath:
    def add(self, node):
        return NonemptyLinkedPath(node, self)

    def visited(self, node):        
        return False

    def len(self):
        return 0

    def as_list(self):
        return []

class NonemptyLinkedPath:
    def __init__(self, head, tail):
        self.head = head
        self.tail = tail
    
    def add(self, node):
        return NonemptyLinkedPath(node, self)
    
    def visited(self, node):
        return node == self.head or self.tail.visited(node)
    
    def len(self):
        return 1 + self.tail.len()

    def as_list(self):
        return self.tail.as_list() + [self.head]

In [ ]:
find_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

In [ ]:
[p.as_list() for p in find_all_paths(graph, 'A', 'D', EmptyLinkedPath())]

In [ ]:
find_shortest_path(graph, 'A', 'D', EmptyLinkedPath()).as_list()

In [ ]:
perf_test(200, EmptyLinkedPath())

##  A Customized Path Data Structure

In [ ]:
perf_test(1000, CustomPath())

In [ ]:
def straight_line_from(fr, n):
    return {i: [i+1] for i in range(fr, fr+n)}

def perf_test_from(fr, n, path):
    x = [find_path(straight_line_from(fr, n), 0, n, path)
         for i in range(1000)]

In [ ]:
perf_test_from(100, 2000, CustomPath())

## Exhaustive Testing for Correct Encapsulation

In [ ]:
class PathLen:
    def call(self, path):
        return path.len()
    def mutates(self):
        return False

class PathAsList:
    def call(self, path):
        return path.as_list()
    def mutates(self):
        return False

class PathAdd:
    def __init__(self, node):
        self.node = node
    
    def call(self, path):
        return path.add(self.node)
    def mutates(self):
        return True

class PathVisited:
    def __init__(self, node):
        self.node = node
    
    def call(self, path):
        return path.visited(self.node)
    def mutates(self):
        return False

In [ ]:
# Generate all method calls whose arguments are positive integers smaller than the bound.
def all_calls(bound):
    return [PathLen(), PathAsList()] \
        + [PathAdd(i) for i in range(bound)] \
        + [PathVisited(i) for i in range(bound)]

# Generate all bounded-call sequences of a given length.
def all_call_seqs(bound, length):
    if length == 0:
        return [[]]
    else:
        return [[call] + calls
                for call in all_calls(bound)
                for calls in all_call_seqs(bound, length-1)]

# Given two path implementations and a list of tests (each a call sequence),
# verify that the two give all the same answers.
def agree_on(path1, path2, tests):
    for calls in tests:
        p1 = path1
        p2 = path2
        # Note that we make copies here into local variables,
        # which we modify as we loop through the current test.

        for call in calls:
            pass

In [ ]:
agree_on(Path(), EmptyLinkedPath(), all_call_seqs(3, 6))

In [ ]:
agree_on(Path(), FancyPath(), all_call_seqs(3, 6))

# Mutable Finite Sets

In [ ]:
def distinct(ls, set):
    for v in ls:
        set.add(v)
    return set.size()

In [ ]:
class NativeSet:
    def __init__(self):
        self.set = set()

    def add(self, v):
        self.set.add(v)

    def mem(self, v):
        return v in self.set

    def size(self):
        return len(self.set)
    
    def as_list(self):
        return sorted(list(self.set))

In [ ]:
distinct([1, 2, 3, 4, 2, 6, 7, 8, 8, 10], NativeSet())

## Unsorted-Linked-List Sets

In [ ]:
class ListNode:
    def __init__(self, head, tail):
        self.head = head
        self.tail = tail

class ListSet:
    def __init__(self):
        self.list = None
    
    def add(self, v):
        if not self.mem(v):
            self.list = ListNode(v, self.list)
    
    def mem(self, v):
        ls = self.list
        while ls != None:
            if ls.head == v:
                return True
            else:
                ls = ls.tail
        return False

    def size(self):
        ls = self.list
        n = 0
        while ls != None:
            n += 1
            ls = ls.tail
        return n

    def as_list(self):
        ls = self.list
        out = []
        while ls != None:
            out.append(ls.head)
            ls = ls.tail
        return sorted(out)

In [ ]:
distinct([1, 2, 3, 4, 2, 6, 7, 8, 8, 10], ListSet())

## Binary-Search-Tree Sets

In [ ]:
class TreeNode:
    def __init__(self, left, value, right):
        self.left = left
        self.value = value
        self.right = right

class TreeSet:
    def __init__(self):
        self.tree = None
    
    def mem(self, v):
        t = self.tree
        while t != None:
            if v == t.value:
                return True
            elif v < t.value:
                t = t.left
            else:
                t = t.right
        return False
    
    def size(self):
        def size_helper(t):
            if t == None:
                return 0
            else:
                return 1 + size_helper(t.left) + size_helper(t.right)
        return size_helper(self.tree)
    
    def add(self, v):
        t = self.tree
        prev = None # What's this variable about?
                    # Watch it get updated below, and then see how it's finally used at the end.
                    # It records where we store a reference to the new node we allocate.
        while t != None:
            if v == t.value:
                return
            elif v < t.value:
                prev = (t, 'left')
                t = t.left
            else:
                prev = (t, 'right')
                t = t.right
        new = TreeNode(None, v, None)
        if prev == None:
            self.tree = new
        elif prev[1] == 'left':
            prev[0].left = new
        else:
            prev[0].right = new
    
    def as_list(self):
        t = self.tree
        out = []
        
        def as_list_helper(t):
            if t != None:
                as_list_helper(t.left)
                out.append(t.value)
                as_list_helper(t.right)
        
        as_list_helper(t)
        return out

In [ ]:
distinct([1, 2, 3, 4, 2, 6, 7, 8, 8, 10], TreeSet())

## Exhaustive Testing

In [ ]:
class SetSize:
    def call(self, set):
        return set.size()
    
class SetAsList:
    def call(self, set):
        return set.as_list()

class SetAdd:
    def __init__(self, v):
        self.v = v
    
    def call(self, set):
        return set.add(self.v)

class SetMem:
    def __init__(self, v):
        self.v = v
    
    def call(self, set):
        return set.mem(self.v)

def all_calls(bound):
    return [SetSize(), SetAsList()] \
        + [SetAdd(i) for i in range(bound)] \
        + [SetMem(i) for i in range(bound)]

def all_call_seqs(bound, length):
    if length == 0:
        return [[]]
    else:
        return [[call] + calls
                for call in all_calls(bound)
                for calls in all_call_seqs(bound, length-1)]

def agree_on(set1, set2, tests):
    for calls in tests:
        s1 = set1()
        s2 = set2()
        # Note that here we allocate new sets at the start of a test!
        # Thus, the arguments set1 and set2 are classes rather than objects.

        for call in calls:
            pass

In [ ]:
agree_on(NativeSet, ListSet, all_call_seqs(3, 6))

In [ ]:
agree_on(NativeSet, TreeSet, all_call_seqs(3, 6))

# Graphs as an Abstract Data Type

In [ ]:
def find_path(graph, start, end, path):
    path = path.add(start)
    if start == end:
        return path
    if not graph.hasNode(start):
        return None
    for node in graph.neighbors(start):
        if not path.visited(node):
            newpath = find_path(graph, node, end, path)
            if newpath: return newpath
    return None

In [ ]:
class BasicGraph:
    def __init__(self):
        self.nodes = []
        self.edges = []
    
    def addNode(self, n):
        if n not in self.nodes:
            self.nodes.append(n)
    
    def addEdge(self, n1, n2):
        if n1 not in self.nodes:
            raise KeyError
        if (n1, n2) not in self.edges:
            self.edges.append((n1, n2))
    
    def hasNode(self, n):
        return n in self.nodes
    
    def neighbors(self, n):
        if n not in self.nodes:
            raise KeyError
        return sorted([n2
                       for n1, n2 in self.edges
                       if n1 == n])
    
    def makeEmpty(self):
        return BasicGraph()

For testing, it will be handy to have an operation to convert our original dictionary-based graph format into whatever format a graph class uses.

In [ ]:
def create_graph(nodes, graph):
    for node, neighbors in nodes.items():
        graph.addNode(node)
        for neighbor in neighbors:
            graph.addEdge(node, neighbor)
    return graph

In [ ]:
find_path(create_graph(graph, BasicGraph()), 'A', 'D', Path()).as_list()

## A Graph Data Type from Any Set Data Type

In [1]:
class GraphUsingSet:
    def __init__(self, setClass):
        self.setClass = setClass
        self.nodes = {}
    
    def addNode(self, n):
        raise NotImplementedError
    
    def addEdge(self, n1, n2):
        if n1 not in self.nodes:
            raise KeyError
        raise NotImplementedError
    
    def hasNode(self, n):
        return n in self.nodes
    
    def neighbors(self, n):
        if n not in self.nodes:
            raise KeyError
        raise NotImplementedError

    def makeEmpty(self):
        return GraphUsingSet(self.setClass)

In [ ]:
find_path(create_graph(graph, GraphUsingSet(NativeSet)), 'A', 'D', Path()).as_list()

In [ ]:
find_path(create_graph(graph, GraphUsingSet(TreeSet)), 'A', 'D', EmptyLinkedPath()).as_list()

## A Very Specific Graph Implementation

In [ ]:
class RatherSpecificGraph:
    def __init__(self):
        raise NotImplementedError
    
    def addNode(self, n):
        raise NotImplementedError
    
    def addEdge(self, n1, n2):
        raise NotImplementedError
    
    def hasNode(self, n):
        raise NotImplementedError
    
    def neighbors(self, n):
        raise NotImplementedError

    def makeEmpty(self):
        raise NotImplementedError

In [ ]:
find_path(create_graph(straight_line(2000), BasicGraph()), 0, 2000, RangePath())

In [ ]:
find_path(RatherSpecificGraph(), 0, 2000, RangePath())

## Exhaustive Testing

In [ ]:
class GraphAddNode:
    def __init__(self, node):
        self.node = node
    
    def call(self, graph):
        return graph.addNode(self.node)

class GraphAddEdge:
    def __init__(self, node1, node2):
        self.node1 = node1
        self.node2 = node2
    
    def call(self, graph):
        return graph.addEdge(self.node1, self.node2)

class GraphHasNode:
    def __init__(self, node):
        self.node = node
    
    def call(self, graph):
        return graph.hasNode(self.node)

class GraphNeighbors:
    def __init__(self, node):
        self.node = node
    
    def call(self, graph):
        return graph.neighbors(self.node)

def all_calls(bound):
    return [GraphAddNode(i) for i in range(bound)] \
        + [GraphAddEdge(i, j) for i in range(bound) for j in range(bound)] \
        + [GraphHasNode(i) for i in range(bound)] \
        + [GraphNeighbors(i) for i in range(bound)]

def all_call_seqs(bound, length):
    if length == 0:
        return [[]]
    else:
        return [[call] + calls
                for call in all_calls(bound)
                for calls in all_call_seqs(bound, length-1)]

def agree_on(graph1, graph2, tests):
    for calls in tests:
        # We forced graphs to support methods to create empty instances of the same class.
        # Why not pass in a class name like before?  It would get messy because we need to pass
        # set implementations to some of our graph classes.
        g1 = graph1.makeEmpty()
        g2 = graph2.makeEmpty()

        for call in calls:
            pass

In [ ]:
agree_on(BasicGraph(), GraphUsingSet(ListSet), all_call_seqs(3, 4))

In [ ]:
agree_on(BasicGraph(), GraphUsingSet(TreeSet), all_call_seqs(3, 4))